In [ ]:
!pip install git+https://github.com/johannfaouzi/pyts.git

In [ ]:
!pip install tensorflow_addons

In [ ]:
import torch
from tqdm import tqdm
import os
import pandas as pd 
import numpy as np
import gc
import sys
import random
import json
import random, string
import seaborn as sns
import matplotlib.pyplot as plt
import logging
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler,LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from scipy import signal
from keras.models import Sequential
from sklearn.model_selection import KFold,StratifiedGroupKFold,StratifiedKFold
from tensorflow.keras.utils import to_categorical
from scipy.signal import savgol_filter
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.fftpack import fft, dct
import tensorflow
from tensorflow.keras.callbacks import *
import scipy
from scipy.stats.mstats import gmean
from scipy.stats import chi2
from matplotlib import patches
import matplotlib.ticker as ticker
import seaborn as sns
import statsmodels.api as sm  
from statsmodels.stats.outliers_influence import variance_inflation_factor
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv1D, AveragePooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import ReLU
import tensorflow_addons as tfa
from keras.layers import Conv1D,Conv2D, Reshape, GaussianNoise

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

In [ ]:
maindir = "." # Directory with your files
traincsv = maindir+"/Update_train.csv"
testcsv = maindir+"/Updated_Test.csv"
sample_sub = maindir+'/Updated_Sample_Submission.csv'
wav_csv = maindir+"/Zindi_Contest_Spectral_Cholesterol.csv"

cholesterol_signature = maindir + "/Zindi Contest Spectra - Cholesterol.csv"
hemoglobin_signature = maindir + "/Zindi Contest Spectra - Hemoglobin.csv"
glucose_signature = maindir + "/Zindi Contest Spectra - Glucose.csv"
skin_fat_blood_signature = maindir + "/Zindi Contest Spectra - skin_fat_blood.csv"

wave_lengths = pd.read_csv(wav_csv)

targets = [
           'hdl_cholesterol_human', 
           'cholesterol_ldl_human', 
           'hemoglobin(hgb)_human'
           ]

In [ ]:
wave_len_info  = wave_lengths['Cholesterol'].values[1:].tolist()
len(wave_len_info)

In [ ]:
waves = []

for x in range(148):
  waves.append(float(wave_len_info[x]))

waves = np.array(waves, dtype = float)
waves.shape

In [ ]:
waves_sig = []

for x in range(170):
  waves_sig.append(float(wave_len_info[x]))

waves_sig = np.array(waves_sig, dtype = float)
waves_sig.shape

In [ ]:
mask_wls_sg = (waves_sig >= 950) & (waves_sig <= 1350)
clipped_waves_sig = waves_sig[mask_wls_sg]
clipped_waves_sig.shape

In [ ]:
mask_wls = (waves >= 950) & (waves <= 1350)
clipped_waves = waves[mask_wls]
clipped_waves.shape

In [ ]:
waves_clipped = clipped_waves.copy()
cols = [str(c) for c in clipped_waves]

In [ ]:
# Loading datasets
train = pd.read_csv(traincsv)
test = pd.read_csv(testcsv)

In [ ]:
train_abs = train[[f'absorbance{i}' for i in range(148)]]
train_abs.columns = wave_len_info[:148]

test_abs = test[[f'absorbance{i}' for i in range(148)]]
test_abs.columns = wave_len_info[:148]

In [ ]:
test.drop([f'absorbance{i}' for i in range(148)], axis=1,  inplace=True)
train.drop([f'absorbance{i}' for i in range(148)], axis=1, inplace=True)

In [ ]:
train = pd.concat([train_abs, train], axis=1)
test =  pd.concat([test_abs,  test], axis=1)

In [ ]:
train.head()

In [ ]:
cholesterol_signature_df = pd.read_csv(cholesterol_signature)
hemoglobin_signature_df = pd.read_csv(hemoglobin_signature)
glucose_signature_df = pd.read_csv(glucose_signature)
skin_fat_blood_signature_df = pd.read_csv(skin_fat_blood_signature)

In [ ]:
hemoglobin_signature_df

In [ ]:
cholesterol_signature_df.rename({'Cholesterol' : 'wavelength', 'Unnamed: 1':'absorbance_coef'},axis=1, inplace=True)
hemoglobin_signature_df.rename({'deox_hb' : 'wavelength', 'Unnamed: 1' :'deoxy_absorbance_coef'},axis=1, inplace=True)
hemoglobin_signature_df.rename({'oxy_hb' : 'wavelength', 'Unnamed: 4' :'oxy_absorbance_coef'},axis=1, inplace=True)
glucose_signature_df.rename({'Glucose' : 'wavelength', 'Unnamed: 1':'absorbance_coef'},axis=1, inplace=True)
skin_fat_blood_signature_df.rename({'Skin' : 'skin_wavelength', 'Unnamed: 1':'skin_absorbance_coef'},axis=1, inplace=True)
skin_fat_blood_signature_df.rename({'Fat' : 'fat_wavelength', 'Unnamed: 4':'fat_absorbance_coef'},axis=1, inplace=True)
skin_fat_blood_signature_df.rename({'Deoxy' : 'deoxy_wavelength', 'Unnamed: 7':'deoxy_absorbance_coef'},axis=1, inplace=True)

In [ ]:
hemoglobin_signature_df.head()

In [ ]:
cholesterol_signature_df.drop(cholesterol_signature_df.index[0], inplace=True)
hemoglobin_signature_df.drop(hemoglobin_signature_df.index[0], inplace=True)
glucose_signature_df.drop(glucose_signature_df.index[0],inplace=True)
skin_fat_blood_signature_df.drop(skin_fat_blood_signature_df.index[0], inplace=True)

In [ ]:
cholesterol_sig = cholesterol_signature_df['absorbance_coef'].values[mask_wls_sg]
deoxy_hemoglobin_sig =  hemoglobin_signature_df['deoxy_absorbance_coef'].values[mask_wls]
oxy_hemoglobin_sig =  hemoglobin_signature_df['oxy_absorbance_coef'].values[mask_wls]
glucose_sig    = glucose_signature_df['absorbance_coef'].values[mask_wls_sg]
skin_sig = skin_fat_blood_signature_df['skin_absorbance_coef'].values[mask_wls_sg]
fat_sig = skin_fat_blood_signature_df['fat_absorbance_coef'].values[mask_wls_sg]
deoxy_sig = skin_fat_blood_signature_df['deoxy_absorbance_coef'].values[mask_wls_sg]

In [ ]:
cholesterol_sig.shape, deoxy_hemoglobin_sig.shape, oxy_hemoglobin_sig.shape, glucose_sig.shape, skin_sig.shape, skin_sig.shape, fat_sig.shape, deoxy_sig.shape

In [ ]:
cholesterol_signature_df

In [ ]:
train = train[train.donation_id != 6824].reset_index(drop=True) # Remove Donation_ID 6824

In [ ]:
train_spectra = train[cols].values
test_spectra = test[cols].values

In [ ]:
cholesterol_sig = np.array(cholesterol_sig, dtype = float)
deoxy_hemoglobin_sig = np.array(deoxy_hemoglobin_sig, dtype = float)
oxy_hemoglobin_sig = np.array(oxy_hemoglobin_sig, dtype = float)
glucose_sig    = np.array(glucose_sig, dtype = float)
skin_sig = np.array(skin_sig, dtype = float)
fat_sig = np.array(fat_sig, dtype = float)
deoxy_sig = np.array(deoxy_sig, dtype = float)

In [ ]:
num_plots = 1

for x in range(num_plots):
  plt.plot(clipped_waves, train_spectra[x, :], 'r')

In [ ]:
from pandas.compat.numpy import np
from scipy import newaxis as nA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, cohen_kappa_score, roc_auc_score
from sklearn.metrics import confusion_matrix, f1_score 


# Important Utils
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def plot_loss(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    for l in loss_list:
            plt.plot(epochs, history.history[l], 'b', label='Training loss (' + 
                     str(str(format(history.history[l][-1],'.5f'))+')'))

class GlobalStandardScaler(object):
    """Scales to unit standard deviation and mean centering using global mean and std of X, skleran like API"""
    def __init__(self,with_mean=True, with_std=True, normfact=1.0):
        self._with_mean = with_mean
        self._with_std = with_std
        self.std = None
        self.normfact=normfact
        self.mean = None
        self._fitted = False
        
    def fit(self,X, y = None):
        X = np.array(X)
        self.mean = X.mean()
        self.std = X.std()
        self._fitted = True
        
    def transform(self,X, y=None):
        if self._fitted:
            X = np.array(X)
            if self._with_mean:
                X=X-self.mean
            if self._with_std:
                X=X/(self.std*self.normfact)
            return X
        else:
            print("Scaler is not fitted")
            return
            
    def inverse_transform(self,X, y=None):
        if self._fitted:
            X = np.array(X)
            if self._with_std:
                X=X*self.std*self.normfact
            if self._with_mean:
                X=X+self.mean
            return X
        else:
            print("Scaler is not fitted")
            return
            
    def fit_transform(self,X, y=None):
        self.fit(X)
        return self.transform(X)

class EmscScaler(object):
    def __init__(self,order=1):
        self.order = order
        self._mx = None
        
    def mlr(self,x,y):
        """Multiple linear regression fit of the columns of matrix x 
        (dependent variables) to constituent vector y (independent variables)
        
        order -     order of a smoothing polynomial, which can be included 
                    in the set of independent variables. If order is
                    not specified, no background will be included.
        b -         fit coeffs
        f -         fit result (m x 1 column vector)
        r -         residual   (m x 1 column vector)
        """
        
        if self.order > 0:
            s= np.ones((len(y),1))
            for j in range(self.order):
                s= np.concatenate((s,(np.arange(0,1+(1.0/(len(y)-1)),1.0/(len(y)-1))**j)[:,nA]),1)
            X= np.concatenate((x, s),1)
        else:
            X = x
        
        #calc fit b=fit coefficients
        b = np.dot(np.dot(np.linalg.pinv(np.dot(np.transpose(X),X)),np.transpose(X)),y)
        f = np.dot(X,b)
        r = y - f

        return b,f,r

    
    def inverse_transform(self, X, y=None):
        print("Warning: inverse transform not possible with Emsc")
        return X
    
    def fit(self, X, y=None):
        """fit to X (get average spectrum), y is a passthrough for pipeline compatibility"""
        self._mx = np.mean(X,axis=0)[:,nA]
        
    def transform(self, X, y=None, copy=None):
        if type(self._mx) == type(None):
            print("EMSC not fit yet. run .fit method on reference spectra")
        else:
            #do fitting
            corr = np.zeros(X.shape)
            for i in range(len(X)):
                b,f,r = self.mlr(self._mx, X[i,:][:,nA])
                corr[i,:] = np.reshape((r/b[0,0]) + self._mx, (corr.shape[1],))
            return corr
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)
          

In [ ]:
def transform_c_ldl(row):
    return str(row["donation_id"]) + "_cholesterol_ldl_human" +  "-" + row["cholesterol_ldl_human"]

def transform_c_hdl(row):
    return str(row["donation_id"]) + "_hdl_cholesterol_human" +  "-" + row["hdl_cholesterol_human"]

def transform_hemo(row):
    return str(row["donation_id"]) + "_hemoglobin(hgb)_human" +  "-" + row["hemoglobin(hgb)_human"]

In [ ]:
test['donation_id'] = test['donation_id'].apply(lambda x : str(x).split('_')[-1])

In [ ]:
data = pd.concat([train, test], ignore_index = True)
data

In [ ]:
drop_waves = [str(c) for c in waves if c not in waves_clipped]
drop_waves[:5]

In [ ]:
del drop_waves[0]

In [ ]:
drop_waves.append('900')

In [ ]:
data.drop(columns = drop_waves, axis = 1, inplace=True)

In [ ]:
test_ = data[data.Reading_ID.isin(test.Reading_ID.values.tolist())].reset_index(drop=True)
train_ = data[~data.Reading_ID.isin(test.Reading_ID.values.tolist())].reset_index(drop=True)

In [ ]:
train_.shape, test_.shape

In [ ]:
train_spectra = train_[cols].values
test_spectra  = test_[cols].values

In [ ]:
train_[cols].head()

In [ ]:
feature_cols_env = ['humidity', 'temperature']

In [ ]:
train_[cols + feature_cols_env] = train_.groupby(["donation_id"])[cols + feature_cols_env].transform("median")
test_[cols + feature_cols_env]  = test_.groupby(["donation_id"])[cols + feature_cols_env].transform("median")

In [ ]:
train_ = train_.drop_duplicates(subset=['donation_id'], keep= 'first')
test_  = test_.drop_duplicates(subset=['donation_id'],  keep= 'first')

In [ ]:
train_.reset_index(drop=True, inplace=True)
test_.reset_index(drop=True, inplace=True)

In [ ]:
X_train = train_[cols].values
X_test = test_[cols].values

In [ ]:
train_

In [ ]:
cholesterol_sig.shape

In [ ]:
num_plots = 1

for x in range(num_plots):
  plt.plot(waves_clipped, cholesterol_sig, 'r')

In [ ]:
train_spec = train_[cols].values
test_spec = test_[cols].values

In [ ]:
train_.drop([c for c in train_.columns if "absorbance" in c], axis = 1, inplace=True)

In [ ]:
test_.drop([c for c in test_.columns if "absorbance" in c], axis = 1, inplace=True)

In [ ]:
df_hdl = train_.drop([targets[1], targets[2]], axis=1)
df_ldl = train_.drop([targets[0], targets[2]], axis=1)
df_hgb = train_.drop([targets[0], targets[1]], axis=1)

In [ ]:
test_.drop(targets , axis=1, inplace=True)

In [ ]:
df_hgb['hemoglobin(hgb)_human'].value_counts()

In [ ]:
df_ldl

In [ ]:
df_hdl_f = df_hdl.copy()
df_ldl_f = df_ldl.copy()
df_hgb_f = df_hgb.copy() 

In [ ]:
df_hdl_f

In [ ]:
df_test_hdl = test_.copy()
df_test_ldl = test_.copy()
df_test_hgb = test_.copy()

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, df_hdl_f[cols].values[x, :], 'r')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, fat_sig, 'r')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, cholesterol_sig, 'r')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, deoxy_hemoglobin_sig, 'r')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, oxy_hemoglobin_sig, 'r')

In [ ]:
df_hgb_f

In [ ]:
cols[:5]

In [ ]:
df_hdl_f.drop([c for c in df_hdl_f.columns if c not in cols + ['id', 'std','donation_id', 'temperature', 'humidity']+ targets], inplace=True, axis=1)
df_ldl_f.drop([c for c in df_ldl_f.columns if c not in cols + ['id', 'std','donation_id', 'temperature', 'humidity']+ targets],inplace=True, axis=1)
df_hgb_f.drop([c for c in df_hgb_f.columns if c not in cols + ['id', 'std','donation_id', 'temperature', 'humidity']+ targets],inplace=True, axis=1)

In [ ]:
cholesterol_sig.min(), cholesterol_sig.max(), cholesterol_sig.shape

In [ ]:
df_ldl_f[cols].values[0, :].min(),  df_ldl_f[cols].values[0, :].max()

In [ ]:
def compute_correlation(whole, sub):

  whole_mean = whole.mean()

  whole = whole - whole_mean
  sub   = sub - whole_mean

  corr = signal.correlate(whole, sub, mode = 'same')

  return corr

In [ ]:
df_hdl_f.shape

**USE CORRELATION TO ISOLATE PURE COMPOUND PATTERNS FROM THE READINGS**

In [ ]:
df_hdl_f_arr = np.array(df_hdl_f[cols].values)
test_hdl_arr = np.array(df_test_hdl[cols].values)
df_ldl_f_arr = np.array(df_ldl_f[cols].values)
df_hgb_f_arr = np.array(df_hgb_f[cols].values)

In [ ]:
df_hdl_f_arr.shape, df_ldl_f.shape, df_hgb_f.shape

In [ ]:
hb_corr_arr = []

for i in range(df_hdl_f_arr.shape[0]):
  arr = compute_correlation(df_hdl_f_arr[i, :].squeeze(), deoxy_hemoglobin_sig)
  hb_corr_arr.append(arr)

hdl_deoxy  = np.array(hb_corr_arr)

In [ ]:
hb_corr_arr = []

for i in range(df_ldl_f_arr.shape[0]):
  arr = compute_correlation(df_ldl_f_arr[i, :].squeeze(), deoxy_hemoglobin_sig)
  hb_corr_arr.append(arr)

ldl_deoxy  = np.array(hb_corr_arr)

In [ ]:
hb_corr_arr = []

for i in range(df_hgb_f_arr.shape[0]):
  arr = compute_correlation(df_hgb_f_arr[i, :].squeeze(), deoxy_hemoglobin_sig)
  hb_corr_arr.append(arr)

hgb_deoxy  = np.array(hb_corr_arr)

In [ ]:
hb_corr_arr = []

for i in range(test_hdl_arr.shape[0]):
  arr = compute_correlation(test_hdl_arr[i, :].squeeze(), deoxy_hemoglobin_sig)
  hb_corr_arr.append(arr)


test_deoxy  = np.array(hb_corr_arr)

In [ ]:
oxy_corr_arr = []

for i in range(df_hdl_f_arr.shape[0]):
  arr = compute_correlation(df_hdl_f_arr[i, :].squeeze(), oxy_hemoglobin_sig)
  oxy_corr_arr.append(arr)

hdl_oxy  = np.array(oxy_corr_arr)

In [ ]:
oxy_corr_arr = []

for i in range(df_ldl_f_arr.shape[0]):
  arr = compute_correlation(df_ldl_f_arr[i, :].squeeze(), oxy_hemoglobin_sig)
  oxy_corr_arr.append(arr)

ldl_oxy  = np.array(oxy_corr_arr)

In [ ]:
oxy_corr_arr = []

for i in range(df_hgb_f_arr.shape[0]):
  arr = compute_correlation(df_hgb_f_arr[i, :].squeeze(), oxy_hemoglobin_sig)
  oxy_corr_arr.append(arr)

hgb_oxy  = np.array(oxy_corr_arr)

In [ ]:
oxy_corr_arr = []

for i in range(test_hdl_arr.shape[0]):
  arr = compute_correlation(test_hdl_arr[i, :].squeeze(), oxy_hemoglobin_sig)
  oxy_corr_arr.append(arr)

test_oxy  = np.array(oxy_corr_arr)

In [ ]:
fat_corr_arr = []

for i in range(df_hdl_f_arr.shape[0]):
  arr = compute_correlation(df_hdl_f_arr[i, :].squeeze(), fat_sig)
  fat_corr_arr.append(arr)

hdl_fat  = np.array(fat_corr_arr)

In [ ]:
fat_corr_arr = []

for i in range(df_ldl_f_arr.shape[0]):
  arr = compute_correlation(df_ldl_f_arr[i, :].squeeze(), fat_sig)
  fat_corr_arr.append(arr)

ldl_fat  = np.array(fat_corr_arr)

In [ ]:
fat_corr_arr = []

for i in range(df_hgb_f_arr.shape[0]):
  arr = compute_correlation(df_hgb_f_arr[i, :].squeeze(), fat_sig)
  fat_corr_arr.append(arr)

hgb_fat  = np.array(fat_corr_arr)

In [ ]:
fat_corr_arr = []

for i in range(test_hdl_arr.shape[0]):
  arr = compute_correlation(test_hdl_arr[i, :].squeeze(), fat_sig)
  fat_corr_arr.append(arr)

test_fat  = np.array(fat_corr_arr)

In [ ]:
glu_corr_arr = []

for i in range(df_hdl_f_arr.shape[0]):
  arr = compute_correlation(df_hdl_f_arr[i, :].squeeze(), glucose_sig)
  glu_corr_arr.append(arr)

hdl_glu  = np.array(glu_corr_arr)

In [ ]:
glu_corr_arr = []

for i in range(df_ldl_f_arr.shape[0]):
  arr = compute_correlation(df_ldl_f_arr[i, :].squeeze(), glucose_sig)
  glu_corr_arr.append(arr)

ldl_glu  = np.array(glu_corr_arr)

In [ ]:
glu_corr_arr = []

for i in range(df_hgb_f_arr.shape[0]):
  arr = compute_correlation(df_hgb_f_arr[i, :].squeeze(), glucose_sig)
  glu_corr_arr.append(arr)

hgb_glu  = np.array(glu_corr_arr)

In [ ]:
glu_corr_arr = []

for i in range(test_hdl_arr.shape[0]):
  arr = compute_correlation(test_hdl_arr[i, :].squeeze(), glucose_sig)
  glu_corr_arr.append(arr)

test_glu  = np.array(glu_corr_arr)

In [ ]:
net_hdl_f_arr = hdl_deoxy - (hdl_fat + hdl_glu)

In [ ]:
net_ldl_f_arr = ldl_deoxy - ldl_glu

In [ ]:
net_hgb_f_arr = hgb_deoxy  - (hgb_fat + hgb_glu)

In [ ]:
df_hdl_f[cols] = net_hdl_f_arr

In [ ]:
df_ldl_f[cols] = net_ldl_f_arr

In [ ]:
df_hgb_f[cols] = net_hgb_f_arr

In [ ]:
net_test_f_arr = test_deoxy - (test_fat + test_glu)

In [ ]:
net_test_ldl = test_deoxy - test_glu

In [ ]:
net_test_hdl =  net_test_f_arr

In [ ]:
df_test_hdl[cols] = net_test_hdl 
df_test_hgb[cols] = net_test_f_arr
df_test_ldl[cols] = net_test_ldl 

**Spectral Preprocessing**

In [ ]:
df_hdl_f['hdl_cholesterol_human'] = df_hdl_f['hdl_cholesterol_human'].map({'low': 0, 'ok': 1, 'high' : 2})
df_hgb_f['hemoglobin(hgb)_human'] = df_hgb_f['hemoglobin(hgb)_human'].map({'low': 0, 'ok': 1, 'high' : 2})
df_ldl_f['cholesterol_ldl_human'] = df_ldl_f['cholesterol_ldl_human'].map({'low': 0, 'ok': 1, 'high' : 2})

In [ ]:
g_scaler1 = GlobalStandardScaler()
emsc_corr1 = EmscScaler()

In [ ]:
hgb_tgt = df_hgb_f['hemoglobin(hgb)_human'].values
hdl_tgt = df_hdl_f['hdl_cholesterol_human'].values
ldl_tgt = df_ldl_f['cholesterol_ldl_human'].values

In [ ]:
hgb_spec = df_hgb_f[cols].values
hdl_spec = df_hdl_f[cols].values
ldl_spec = df_ldl_f[cols].values

In [ ]:
hgb_scaled = g_scaler1.fit_transform(hgb_spec, hgb_tgt)
hgb_corrected = emsc_corr1.fit_transform(hgb_scaled,hgb_tgt)
df_hgb_f[cols] = hgb_corrected

In [ ]:
g_scaler2 = GlobalStandardScaler()
emsc_corr2 = EmscScaler()

hdl_scaled = g_scaler2.fit_transform(hdl_spec, hdl_tgt)
hdl_corrected = emsc_corr2.fit_transform(hdl_scaled, hdl_tgt)
df_hdl_f[cols] = hdl_corrected

In [ ]:
g_scaler3 = GlobalStandardScaler()
emsc_corr3 = EmscScaler()

ldl_scaled = g_scaler3.fit_transform(ldl_spec, ldl_tgt)
ldl_corrected = emsc_corr3.fit_transform(ldl_scaled, ldl_tgt)
df_ldl_f[cols] = ldl_corrected

In [ ]:
hgb_env = df_hgb_f[feature_cols_env].values
hdl_env = df_hdl_f[feature_cols_env].values
ldl_env = df_ldl_f[feature_cols_env].values

In [ ]:
df_ldl_f

In [ ]:
hgb_smoothed = savgol_filter(df_hgb_f[cols], window_length=5, polyorder=2, deriv=1, axis = 0)
df_hgb_f[cols] = hgb_smoothed

In [ ]:
hdl_smoothed = savgol_filter(df_hdl_f[cols], window_length=5, polyorder=2, deriv=1, axis = 0)
df_hdl_f[cols] = hdl_smoothed

In [ ]:
ldl_smoothed = savgol_filter(df_ldl_f[cols], window_length=5, polyorder=2, deriv=1, axis = 0)
df_ldl_f[cols] = ldl_smoothed

In [ ]:
hgb_spec_t = df_test_hgb[cols].values
hdl_spec_t = df_test_hdl[cols].values
ldl_spec_t = df_test_ldl[cols].values

In [ ]:
test_scaled = g_scaler1.transform(hgb_spec_t)
test_corrected = emsc_corr1.transform(test_scaled)
df_test_hgb[cols] = test_corrected

In [ ]:
test_scaled = g_scaler2.transform(hdl_spec_t)
test_corrected = emsc_corr2.transform(test_scaled)
df_test_hdl[cols] = test_corrected

In [ ]:
test_scaled = g_scaler3.transform(ldl_spec_t)
test_corrected = emsc_corr3.transform(test_scaled)
df_test_ldl[cols] = test_corrected

In [ ]:
test_smoothed = savgol_filter(df_test_hdl[cols], window_length=5, polyorder=2, deriv=1, axis = 0)
df_test_hdl[cols] = test_smoothed

In [ ]:
test_smoothed = savgol_filter(df_test_ldl[cols], window_length=5, polyorder=2, deriv=1, axis = 0)
df_test_ldl[cols] = test_smoothed

In [ ]:
test_smoothed = savgol_filter(df_test_hgb[cols], window_length=5, polyorder=2, deriv=1, axis = 0)
df_test_hgb[cols] = test_smoothed

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, hgb_smoothed[x, :], 'r')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, ldl_smoothed[x, :], 'b')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, hdl_smoothed[x, :], 'b')

In [ ]:
num_plots = 15

for x in range(num_plots):
  plt.plot(waves_clipped, test_smoothed[x, :], 'b')

In [ ]:
_ = gc.collect()

In [ ]:
len(cols)

In [ ]:
df_hdl_f['sg_std'] = df_hdl_f[cols].apply(lambda x : np.std(x.values), axis=1)
df_ldl_f['sg_std'] = df_ldl_f[cols].apply(lambda x : np.std(x.values), axis=1)
df_hgb_f['sg_std'] = df_hgb_f[cols].apply(lambda x : np.std(x.values), axis=1)

In [ ]:
df_test_hdl['sg_std'] = df_test_hdl[cols].apply(lambda x : np.std(x.values), axis=1)
df_test_ldl['sg_std'] = df_test_ldl[cols].apply(lambda x : np.std(x.values), axis=1)
df_test_hgb['sg_std'] = df_test_hgb[cols].apply(lambda x : np.std(x.values), axis=1)

In [ ]:
df_hdl_f

In [ ]:
feature_cols = cols
feature_cols_env = ['temperature' , 'humidity', 'sg_std']

In [ ]:
new_data_train = pd.concat([df_hdl_f[cols], df_ldl_f[cols], df_hgb_f[cols]], ignore_index=True)
new_data_test  = pd.concat([df_test_hdl[cols], df_test_ldl[cols], df_test_hgb[cols]], ignore_index=True)

In [ ]:
all_data = pd.concat([new_data_train, new_data_test], ignore_index=True)
all_data.head()

In [ ]:
# Finding optimal no. of clusters
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist


clusters = range(1,10)
meanDistortions  = []

for k in clusters:
    model=KMeans(n_clusters=k)
    model.fit(all_data)
    prediction=model.predict(all_data)
    meanDistortions.append(sum(np.min(cdist(all_data, model.cluster_centers_, 'euclidean'), axis=1)) / all_data.shape[0])


plt.plot(clusters, meanDistortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Average distortion')
plt.title('Selecting k with the Elbow Method')

In [ ]:
def fe_cluster(train, test, features, n_clusters= 2, SEED=42):

    def create_cluster(train, test, features, n_clusters=n_clusters):

        train = train.fillna(0)
        test =  test.fillna(0)
        
        train_ = train[features].copy()
        test_ = test[features].copy()

        kmeans = MiniBatchKMeans(n_clusters=n_clusters, max_iter=300, batch_size=32, random_state= SEED)
                
        kmeans.fit(pd.concat((train_, test_), axis=0).reset_index(drop=True))

        train['clusters'] = kmeans.predict(train_.values)
        test['clusters']  = kmeans.predict(test_.values)

        return train, test

    train, test = create_cluster(
        train, test, features, n_clusters=n_clusters)
    
    return train, test

In [ ]:
df_ldl_f, df_test_ldl = fe_cluster(df_ldl_f, df_test_ldl, n_clusters = 2, features = feature_cols)

In [ ]:
df_hdl_f, df_test_hdl = fe_cluster(df_hdl_f, df_test_hdl, n_clusters =  2, features = feature_cols)

In [ ]:
df_hgb_f, df_test_hgb = fe_cluster(df_hgb_f, df_test_hgb, n_clusters = 2, features = feature_cols)

In [ ]:
metrics.fowlkes_mallows_score(df_hdl_f['hdl_cholesterol_human'],df_hdl_f['clusters']) # evaluate hdl_cholesterol_human clusters

In [ ]:
from scipy.stats import zscore

In [ ]:
df_hdl_f[cols] = df_hdl_f[cols].apply(zscore)
df_hdl_f[cols] = df_hdl_f[cols].apply(zscore)
df_hdl_f[cols] = df_hdl_f[cols].apply(zscore)

In [ ]:
for x in cols:

  for df in [df_hdl_f, df_ldl_f, df_hgb_f]:

    indexNames_larger = df[df[x]>3].index
    indexNames_lesser = df[df[x]<-3].index
    # Delete these row indexes from dataFrame
    
    df.drop(indexNames_larger , inplace=True)
    df.drop(indexNames_lesser , inplace=True)

In [ ]:
df_hdl_f.shape, df_ldl_f.shape, df_hgb_f.shape

In [ ]:
df_hdl.shape, df_ldl.shape, df_hgb.shape

In [ ]:
df_hdl_f.reset_index(drop=True, inplace=True)

In [ ]:
x = df_hgb_f.copy()
t = df_test_hgb.copy()

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(x[cols])
T_pca = pca.transform(t[cols])

pca_cols = [f"PC{i+1}" for i in range(X_pca.shape[1])]

X_pca = pd.DataFrame(X_pca, columns=pca_cols, index=df_hgb_f.index)
T_pca = pd.DataFrame(T_pca, columns=pca_cols, index=df_test_hgb.index)

df_hgb_f = pd.concat([df_hgb_f, X_pca], axis=1)
df_test_hgb = pd.concat([df_test_hgb, T_pca], axis=1)

In [ ]:
df_hgb_f

In [ ]:
# PCA Viz

plt.figure(figsize=(15,8))
sns.scatterplot(data=df_hgb_f, x="PC1", y="PC2", hue='hemoglobin(hgb)_human', alpha=0.8, palette="deep")
plt.show()

In [ ]:
x = df_hdl_f.copy()
t = df_test_hdl.copy()

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(x[cols])
T_pca = pca.transform(t[cols])

pca_cols = [f"PC{i+1}" for i in range(X_pca.shape[1])]

X_pca = pd.DataFrame(X_pca, columns=pca_cols, index=df_hdl_f.index)
T_pca = pd.DataFrame(T_pca, columns=pca_cols, index=df_test_hdl.index)

df_hdl_f = pd.concat([df_hdl_f, X_pca], axis=1)
df_test_hdl = pd.concat([df_test_hdl, T_pca], axis=1)


In [ ]:
# PCA Viz

plt.figure(figsize=(15,8))
sns.scatterplot(data=df_hdl_f, x="PC1", y="PC2", hue='hdl_cholesterol_human', alpha=0.8, palette="deep")
plt.show()

In [ ]:
x = df_ldl_f.copy()
t = df_test_ldl.copy()

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(x[cols])
T_pca = pca.transform(t[cols])

pca_cols = [f"PC{i+1}" for i in range(X_pca.shape[1])]

X_pca = pd.DataFrame(X_pca, columns=pca_cols, index=df_ldl_f.index)
T_pca = pd.DataFrame(T_pca, columns=pca_cols, index=df_test_ldl.index)

df_ldl_f = pd.concat([df_ldl_f, X_pca], axis=1)
df_test_ldl = pd.concat([df_test_ldl, T_pca], axis=1)

In [ ]:
# PCA Viz

plt.figure(figsize=(15,8))
sns.scatterplot(data=df_ldl_f, x="PC1", y="PC2", hue='cholesterol_ldl_human', alpha=0.8, palette="deep")
plt.show()

In [ ]:
del pca
_  = gc.collect()

In [ ]:
feature_cols_env = ['humidity', 'temperature', 'clusters', 'sg_std'] + pca_cols

In [ ]:
len(feature_cols_env)

In [ ]:
df_hdl_f.shape, df_ldl_f.shape, df_hgb_f.shape

In [ ]:
feature_cols_env.remove('clusters')

In [ ]:
sc = StandardScaler()
sc.fit(pd.concat([
                  df_hdl_f[feature_cols_env],
                  df_ldl_f[feature_cols_env],
                  df_hgb_f[feature_cols_env],
                  df_test_hdl[feature_cols_env],
                  df_test_ldl[feature_cols_env],
                  df_test_hgb[feature_cols_env]
], ignore_index=True))

df_hdl_f[feature_cols_env] = sc.transform(df_hdl_f[feature_cols_env])
df_ldl_f[feature_cols_env] = sc.transform(df_ldl_f[feature_cols_env])
df_hgb_f[feature_cols_env] = sc.transform(df_hgb_f[feature_cols_env])

df_test_hdl[feature_cols_env] = sc.transform(df_test_hdl[feature_cols_env])
df_test_ldl[feature_cols_env] = sc.transform(df_test_ldl[feature_cols_env])
df_test_hgb[feature_cols_env] = sc.transform(df_test_hgb[feature_cols_env])

In [ ]:
df_hdl_f = pd.get_dummies(df_hdl_f, prefix='cluster', columns = ['clusters'])
df_ldl_f = pd.get_dummies(df_ldl_f, prefix='cluster', columns = ['clusters'])
df_hgb_f = pd.get_dummies(df_hgb_f, prefix='cluster', columns = ['clusters'])

In [ ]:
df_test_hdl = pd.get_dummies(df_test_hdl, prefix='cluster', columns = ['clusters'])
df_test_ldl = pd.get_dummies(df_test_ldl, prefix='cluster', columns = ['clusters'])
df_test_hgb = pd.get_dummies(df_test_hgb, prefix='cluster', columns = ['clusters'])

In [ ]:
df_test_ldl

In [ ]:
df_hdl_f.shape

In [ ]:
feature_cols_env = ['temperature',	'humidity',	'PC1',	'PC2',	'cluster_0',	'cluster_1', 'sg_std']

# **Model Training Utils**

In [ ]:
# 2D CNN Model Training Util
def train_model_NN(data, test_data, n_folds):

  preds = []

  df = data.copy()

  oof_acc = []

  folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state = 42)

  for fold_ , (trn_idx, val_idx) in enumerate(folds.split(df, df[tgt])):

    print('Fold:',fold_)

    if tgt == 'hdl_cholesterol_human':

  
      train_data = df.loc[trn_idx].reset_index(drop=True)
      val_data   = df.loc[val_idx].reset_index(drop=True)

      x_train_p, Y_train = train_data[feature_cols].copy(), train_data[tgt].values

      x_test_p, Y_test = val_data[feature_cols].copy(), val_data[tgt].values

      x_train_s, x_test_s = train_data[feature_cols_env].copy(), val_data[feature_cols_env].copy()

      x_test_data, x_test_aux = test_data[feature_cols].copy(), test_data[feature_cols_env].copy()

      y_train_ = Y_train.copy()

      y_train_ = to_categorical(y_train_ , num_classes = len(targets))

      X_train_f = x_train_p.copy() 

      x_train_imgs = make_image(X_train_f.values)
      x_valid_imgs = make_image(x_test_p.values)
      x_test_imgs = make_image(x_test_data.values)


    elif tgt == 'cholesterol_ldl_human':


      train_data = df.loc[trn_idx].reset_index(drop=True)
      val_data   = df.loc[val_idx].reset_index(drop=True)

      x_train_p, Y_train = train_data[feature_cols].copy(), train_data[tgt].values

      x_test_p, Y_test = val_data[feature_cols].copy(), val_data[tgt].values

      x_train_s, x_test_s = train_data[feature_cols_env].copy(), val_data[feature_cols_env].copy()

      x_test_data, x_test_aux = test_data[feature_cols].copy(), test_data[feature_cols_env].copy()


      y_train_ = Y_train.copy()

      y_train_ = to_categorical(y_train_ , num_classes = len(targets))

      X_train_f = x_train_p.copy() 

      x_train_imgs = make_image(X_train_f.values)
      x_valid_imgs = make_image(x_test_p.values)
      x_test_imgs = make_image(x_test_data.values)

    else:

      
      train_data = df.loc[trn_idx].reset_index(drop=True)
      val_data   = df.loc[val_idx].reset_index(drop=True)

      x_train_p, Y_train = train_data[feature_cols].copy(), train_data[tgt].values

      x_test_p, Y_test = val_data[feature_cols].copy(), val_data[tgt].values

      x_train_s, x_test_s = train_data[feature_cols_env].copy(), val_data[feature_cols_env].copy()

      x_test_data, x_test_aux = test_data[feature_cols].copy(), test_data[feature_cols_env].copy()

      y_train_ = Y_train.copy()

      y_train_ = to_categorical(y_train_ , num_classes = len(targets))

      X_train_f = x_train_p.copy()  

      x_train_imgs = make_image(X_train_f.values)
      x_valid_imgs = make_image(x_test_p.values)
      x_test_imgs = make_image(x_test_data.values)

    y_test_  = Y_test.copy()

    y_test_  = to_categorical(y_test_,  num_classes = len(targets))

    X_valid_f = x_valid_imgs.copy() 
    X_test_f  = x_test_imgs.copy()

    train_dataset = tf.data.Dataset.from_tensor_slices(((x_train_imgs, x_train_s.values), y_train_))
    train_dataset = train_dataset.shuffle(len(y_train_))  
    train_dataset = train_dataset.batch(128).cache()
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    valid_dataset = tf.data.Dataset.from_tensor_slices(((X_valid_f, x_test_s.values), y_test_))
    valid_dataset = valid_dataset.batch(128).cache()
    valid_dataset = valid_dataset.prefetch(tf.data.experimental.AUTOTUNE)


    del x_train_imgs, x_valid_imgs
    _ =gc.collect()


    model = get_spectra_vgg(n_wavenumbers= X_valid_f.shape[1],
                            n_features=3, compiled=True)


    cb_es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                            patience= 20, mode="min", 
                                            restore_best_weights=True, verbose=1)
    
    cb_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                                                patience=10, mode="min",
                                                min_lr=0.00005, verbose=1)

    
    history = model.fit(train_dataset, 
                        validation_data =valid_dataset, 
                        callbacks = [cb_es, cb_lr],
                        epochs = 1000) 



    plot_loss(history)

    del history
    _ = gc.collect()

    y_pre = model.predict([X_valid_f, x_test_s.values])

    y_pred = np.argmax(y_pre, axis=-1)

    score = accuracy_score(Y_test, y_pred) 

    oof_acc.append(score)

    tt_preds = model.predict([x_test_imgs, x_test_aux.values])

    preds.append(tt_preds)


    plt.show()
    plt.scatter(Y_test, y_pred)
    plt.show()
    print("OOF Accuracy: ",score)

  print(f"{tgt} mean accuracy {np.mean(oof_acc)}")
  
  return preds

In [ ]:
targets

In [ ]:
# ref - https://github.com/devitrylouis/imaging_time_series

def tabulate(x, y, f):
    """Return a table of f(x, y). Useful for the Gram-like operations."""
    return np.vectorize(f)(*np.meshgrid(x, y, sparse=True))

def cos_sum(a, b):
    """To work with tabulate."""
    return(math.cos(a+b))

def sin_diff(a, b):
    """To work with tabulate."""
    return(math.sin(a-b))

def create_time_serie(size, time):
    """Generate a time serie of length size and dynamic with respect to time."""
    # Generating time-series
    support = np.arange(0, size)
    serie = np.cos(support + float(time))
    return(t, serie)

In [ ]:
import math

# ref - https://github.com/devitrylouis/imaging_time_series

class GASF:

    def __init__(self):
        pass
    def transform(self, serie):
        """Compute the Gramian Angular Field of an image"""
        # Min-Max scaling
        min_ = np.amin(serie)
        max_ = np.amax(serie)
        scaled_serie = (2*serie - max_ - min_)/(max_ - min_)

        # Floating point inaccuracy!
        scaled_serie = np.where(scaled_serie >= 1., 1., scaled_serie)
        scaled_serie = np.where(scaled_serie <= -1., -1., scaled_serie)

        # Polar encoding
        phi = np.arccos(scaled_serie)
        # Note! The computation of r is not necessary
        r = np.linspace(0, 1, len(scaled_serie))

        # GAF Computation (every term of the matrix)
        gaf = tabulate(phi, phi, cos_sum)

        return(gaf, phi, r, scaled_serie)
    
class GADF:

    def __init__(self):
        pass
    def transform(self, serie):
        """Compute the Gramian Angular Field of an image"""
        # Min-Max scaling
        min_ = np.amin(serie)
        max_ = np.amax(serie)
        scaled_serie = (2*serie - max_ - min_)/(max_ - min_)

        # Floating point inaccuracy!
        scaled_serie = np.where(scaled_serie >= 1., 1., scaled_serie)
        scaled_serie = np.where(scaled_serie <= -1., -1., scaled_serie)

        # Polar encoding
        phi = np.arccos(scaled_serie)
        # Note! The computation of r is not necessary
        r = np.linspace(0, 1, len(scaled_serie))

        # GAF Computation (every term of the matrix)
        gaf = tabulate(phi, phi, sin_diff)

        return(gaf, phi, r, scaled_serie)

In [ ]:
from keras.layers import *

def get_spectra_vgg(n_wavenumbers, n_features=1, compiled=True):

    filter_size = 3
    filters_cnt = 32
    pooling = AveragePooling2D

    inp = Input(shape=(n_wavenumbers, n_wavenumbers, n_features))
    aux_inp = Input(shape=(len(feature_cols_env),))

    conv1 = Conv2D(filters_cnt*2, filter_size, activation='softmax')(inp)
    bn1 = Dropout(0.2)(conv1)

    conv2 = Conv2D(filters_cnt*4, filter_size, activation='softmax')(bn1)
    bn2 = Dropout(0.2)(conv2)

    pool1 = MaxPool2D(pool_size=(2,2))(bn2)

    pool_n = pool1
    filter_mult = 2
    n_blocks = 3
    for i in range(n_blocks):
        conv_n = Conv2D(filters_cnt * filter_mult, filter_size, activation='softmax')(pool_n)
        bn_n = Dropout(0.2)(conv_n)

        conv_n2 = Conv2D(filters_cnt * filter_mult, filter_size, activation='softmax')(bn_n)
        bn_n2 = Dropout(0.2)(conv_n2)

        pool_n = MaxPool2D(pool_size=(2,2))(bn_n2)
        filter_mult *= 2

    x2 = Dense(32, activation='relu')(aux_inp)
    x2 = Dense(64, activation='relu')(x2)

    output = Flatten()(pool_n)

    cat = tf.keras.layers.Concatenate()([x2, output])

    dropout = Dropout(0.25)(cat)
    out = Dense(3 , activation='softmax')(dropout)

    model = Model(inputs=[inp, aux_inp], outputs=out)

    if compiled:
              
        model.compile(loss= 'categorical_crossentropy',
                      optimizer='adam',
                      metrics=[tf.keras.metrics.Recall(name='recall'),
                               tf.keras.metrics.Precision(name='precision'), 
                               'accuracy'] )
        
    return model

**Signal-to-2d-image Utils**

In [ ]:
from pyts.image import MarkovTransitionField
from pyts.datasets import load_gunpoint
import warnings
warnings.filterwarnings("ignore")


def make_gadf_image(x_arr):

  counter = 0
  gadf = GADF()

  x_train_gadf_images = np.zeros((x_arr.shape[0],x_arr.shape[1],x_arr.shape[1]))
  for i in x_arr:
    img = gadf.transform(i)
    x_train_gadf_images[counter] = img[0]

    counter = counter + 1
    
  return x_train_gadf_images

def make_gasf_image(x_arr):

  gasf = GASF()

  counter = 0

  x_train_gasf_images = np.zeros((x_arr.shape[0],x_arr.shape[1],x_arr.shape[1]))
  for i in x_arr:
    img = gasf.transform(i)
    x_train_gasf_images[counter] = img[0]

    counter = counter + 1
    
  return x_train_gasf_images

def apply_markov_transition(x_arr):

  mtf = MarkovTransitionField(image_size=x_arr.shape[-1])
  x_train_mtf_images = np.zeros((x_arr.shape[0],x_arr.shape[-1],x_arr.shape[-1]))
  counter = 0
  for i in x_arr:
      img = mtf.fit_transform(np.transpose(i.reshape(-1,1)))
      x_train_mtf_images[counter] = img[0]
      counter = counter + 1

  return x_train_mtf_images

def make_image(x_arr):

  gadf_img = make_gasf_image(x_arr)
  gasf_img = make_gadf_image(x_arr)
  mtf_img  = apply_markov_transition(x_arr)

  img = np.concatenate((np.expand_dims(gasf_img, axis=3),
                        np.expand_dims(gadf_img, axis=3),
                        np.expand_dims(mtf_img, axis=3)), axis=3)

  return img

**TRAIN DIFFERENT MODELS**

In [ ]:
seed_everything(42)

tgt = 'cholesterol_ldl_human'

pred_ldl = train_model_NN(df_ldl_f, df_test_ldl, n_folds = 10)

In [ ]:
seed_everything(42)

tgt = 'hdl_cholesterol_human'

pred_hdl = train_model_NN(df_hdl_f, df_test_hdl, n_folds = 10)

In [ ]:
seed_everything(42)

tgt = 'hemoglobin(hgb)_human'

pred_hgb = train_model_NN(df_hgb_f, df_test_hgb, n_folds = 10)

In [ ]:
pred_hdl_ = gmean(pred_hdl, 0)
pred_ldl_ = gmean(pred_ldl, 0)
pred_hgb_ = gmean(pred_hgb, 0)

In [ ]:
predictions = np.zeros((test_.shape[0], len(targets)))
predictions[:, 0] = pred_hdl_.argmax(-1)
predictions[:, 1] = pred_ldl_.argmax(-1)
predictions[:, 2] = pred_hgb_.argmax(-1)

In [ ]:
def transform(values):
  labels = []

  for value in values:
    if value == 0:
        labels.append('low')
    elif value == 1:
        labels.append('ok')
    elif value == 2:
        labels.append('high')

  return labels

In [ ]:
df_test_hdl[targets[0]] = predictions[:, 0]
df_test_ldl[targets[1]] = predictions[:, 1]
df_test_hgb[targets[2]] = predictions[:, 2]

In [ ]:
sub_hdl = df_test_hdl[['donation_id', targets[0]]].copy()
sub_ldl = df_test_ldl[['donation_id', targets[1]]].copy()
sub_hgb = df_test_hgb[['donation_id', targets[2]]].copy()

In [ ]:
sub_df = sub_hdl.merge(sub_ldl, on = 'donation_id', how = 'left')
sub_df = sub_df.merge(sub_hgb, on = 'donation_id', how = 'left')

In [ ]:
sub_df[targets] = sub_df[targets].astype(int)

In [ ]:
sub_df[targets] = sub_df[targets].apply(lambda x: transform(x.values), axis=0) 

In [ ]:
final1_class_map = dict(zip(sub_df['donation_id'], sub_df[targets[0]]))
final2_class_map = dict(zip(sub_df['donation_id'], sub_df[targets[1]]))
final3_class_map = dict(zip(sub_df['donation_id'], sub_df[targets[2]]))

In [ ]:
test[targets[0]] = test['donation_id'].map(final1_class_map)
test[targets[1]] = test['donation_id'].map(final2_class_map)
test[targets[2]] = test['donation_id'].map(final3_class_map)

In [ ]:
test_sub = test.copy()

In [ ]:
test_sub.shape

In [ ]:
test_sub

In [ ]:
test_sub[targets] = test_sub[targets].astype(str) 

In [ ]:
test_sub['donation_id'] = test_sub['donation_id'].astype(str)

In [ ]:
hdl_rows = pd.DataFrame(test_sub[['donation_id'] +  targets].apply(transform_c_hdl, axis=1))
hemo_rows = pd.DataFrame(test_sub[['donation_id'] + targets].apply(transform_hemo, axis=1))
ldl_rows = pd.DataFrame(test_sub[['donation_id'] +  targets].apply(transform_c_ldl, axis=1))

In [ ]:
ss = pd.concat([hdl_rows, hemo_rows, ldl_rows]).reset_index(drop=True)

In [ ]:
ss["target"] = ss[0].apply(lambda x: x.split("-")[1])
ss[0] = ss[0].apply(lambda x: x.split("-")[0])

In [ ]:
ss = ss.rename(columns={0:"Donation_ID"})

In [ ]:
ss['Donation_ID'] = 'ID_' + ss['Donation_ID']
ss

In [ ]:
ss.target.value_counts()

In [ ]:
ss.to_csv("submission.csv", index=False)